In [1]:
import pandas as pd

from pli import generate_pli, pli_intersection_optimized
from candidates import prepare_Fk, generate_candidates

Make a toy dataset to play with

In [2]:
data = {
    'a': ['C', 'A', 'A', 'B', 'C'],
    'b': [3, 1, 1, 1, 1],
    'c': ['X', 'X', 'Y', 'X', 'X'],
    'd': [722, 289, 189, 289, 289],
    'e': [112, 553, 583, 513 ,553]
}

df = pd.DataFrame(data)
df

,a,b,c,d,e
0,C,3,X,722,112
1,A,1,X,289,553
2,A,1,Y,189,583
3,B,1,X,289,513
4,C,1,X,289,553


In [122]:
generate_pli(df['a'])

{1: [0, 4], 2: [1, 2]}

# Candidates generation

<div>
<img src="img/candidates_generation.png" width="500"/>
</div>

In [22]:
def prepare_Fk(df):
    """
    Trasform df into the format
    Fk = [(f1, pli1), (f2, pli2), ..., (fk, plik)]
    """
    Fk = []

    for col in df.columns:
        pli = generate_pli(df[col])
        Fk.append((col, pli))

    return Fk


def generate_candidates(Fk):
    """
    Generates candidates for UCC (unique column combination) 
    discovery in Apriori algorithm.

    Input: 
    Fk is a list of non-unique candidates with its PLIs in the 
    following format: Fk = [(f1, pli1), (f2, pli2), ..., (fk, plik)]
    
    Output: 
    Candidates for the next level in the same format as Fk.
    """
    E = []

    schema = [f[0] for f in Fk]

    # traverse all pairs of non-unique column combinations 
    # (if column is unique, than all its supersets will be also unique!) 
    # that share the same maximal prefix (they are in lexicographic order) 
    # and differ only in one last attribute

    for f1, pli1 in Fk:
        for f2, pli2 in Fk:

            # use lexicographic order
            # so consider 2nd element in a pair
            # that is located on the right
            if f1 < f2:

                # check if f1 and f2 share the same maximal prefix 
                # and differ only in the last attribute
                if f1[:-1] == f2[:-1]:

                    # make new CC with k+1 attributes 
                    # using a union of f1 and f2
                    f = f1 + f2[-1]

                    # and make an intersection of PLIs
                    pli = pli_intersection_optimized(pli1, pli2)

                    # if all subsets with k attributes are non-unique
                    if all(''.join(sorted(set(f) - set(i))) in schema for i in f):
                        E.append((''.join(f), pli))             
                               
    return E

In [23]:
E1 = prepare_Fk(df)
E1

[('a', {1: [0, 4], 2: [1, 2]}),
 ('b', {2: [1, 2, 3, 4]}),
 ('c', {1: [0, 1, 3, 4]}),
 ('d', {2: [1, 3, 4]}),
 ('e', {2: [1, 4]})]

In [24]:
E2 = generate_candidates(E1)
E2

[('ab', {(2, 2): [1, 2]}),
 ('ac', {(1, 1): [0, 4]}),
 ('ad', {}),
 ('ae', {}),
 ('bc', {(2, 1): [1, 3, 4]}),
 ('bd', {(2, 2): [1, 3, 4]}),
 ('be', {(2, 2): [1, 4]}),
 ('cd', {(1, 2): [1, 3, 4]}),
 ('ce', {(1, 2): [1, 4]}),
 ('de', {(2, 2): [1, 4]})]

In [25]:
E3 = generate_candidates(E2)
E3

[('abc', {}),
 ('abd', {}),
 ('abe', {}),
 ('acd', {}),
 ('ace', {}),
 ('ade', {}),
 ('bcd', {((2, 1), (2, 2)): [1, 3, 4]}),
 ('bce', {((2, 1), (2, 2)): [1, 4]}),
 ('bde', {((2, 2), (2, 2)): [1, 4]}),
 ('cde', {((1, 2), (1, 2)): [1, 4]})]

In [26]:
E4 = generate_candidates(E3)
E4

[('abcd', {}),
 ('abce', {}),
 ('abde', {}),
 ('acde', {}),
 ('bcde', {(((2, 1), (2, 2)), ((2, 1), (2, 2))): [1, 4]})]

In [123]:
E5 = generate_candidates(E4)
E5

[('abcde', {})]

# Pruning

<div>
<img src="img/pruning.png" width="500"/>
</div>

In [118]:
def prune(E):

    # initialise the set of non-unique candidates
    F = []

    # traverse the candidates
    for f, pli in E:

        # if candidate's pli is NOT empty, 
        # we will add this candidate to the
        # non-unique candidaates list

        if len(pli) > 0:
            F.append((f, pli))
    
    return F

In [124]:
E4

[('abcd', {}),
 ('abce', {}),
 ('abde', {}),
 ('acde', {}),
 ('bcde', {(((2, 1), (2, 2)), ((2, 1), (2, 2))): [1, 4]})]

In [125]:
prune(E4)

[('bcde', {(((2, 1), (2, 2)), ((2, 1), (2, 2))): [1, 4]})]

# Apriori

<div>
<img src="img/apriori_ucc.png" width="500"/>
</div>

In [120]:
def apriori_ucc(df):

    UCC = []
    
    k = 1

    # start with single attribute sets and PLIs
    E = prepare_Fk(df=df)
    F = prune(E)
    UCC.extend(list(sorted(
        set([cand for cand, _ in E]) - set([cand for cand, _ in F])
    )))

    print(f'layer={k}')
    print(f'E={E}') # candidates
    print(f'F={F}') # candidates after pruning
    print()

    while len(F) > 0:

        k += 1

        E_next = generate_candidates(F)
        F_next = prune(E_next)
        F = F_next

        print(f'layer={k}')
        print(f'E={E_next}') # candidates
        print(f'F={F_next}') # candidates after pruning
        print()

        UCC.extend(list(sorted(
            set([cand for cand, _ in E_next]) - set([cand for cand, _ in F_next])
        )))

    return UCC

In [127]:
ucc = apriori_ucc(df)
ucc

layer=1
E=[('a', {1: [0, 4], 2: [1, 2]}), ('b', {2: [1, 2, 3, 4]}), ('c', {1: [0, 1, 3, 4]}), ('d', {2: [1, 3, 4]}), ('e', {2: [1, 4]})]
F=[('a', {1: [0, 4], 2: [1, 2]}), ('b', {2: [1, 2, 3, 4]}), ('c', {1: [0, 1, 3, 4]}), ('d', {2: [1, 3, 4]}), ('e', {2: [1, 4]})]

layer=2
E=[('ab', {(2, 2): [1, 2]}), ('ac', {(1, 1): [0, 4]}), ('ad', {}), ('ae', {}), ('bc', {(2, 1): [1, 3, 4]}), ('bd', {(2, 2): [1, 3, 4]}), ('be', {(2, 2): [1, 4]}), ('cd', {(1, 2): [1, 3, 4]}), ('ce', {(1, 2): [1, 4]}), ('de', {(2, 2): [1, 4]})]
F=[('ab', {(2, 2): [1, 2]}), ('ac', {(1, 1): [0, 4]}), ('bc', {(2, 1): [1, 3, 4]}), ('bd', {(2, 2): [1, 3, 4]}), ('be', {(2, 2): [1, 4]}), ('cd', {(1, 2): [1, 3, 4]}), ('ce', {(1, 2): [1, 4]}), ('de', {(2, 2): [1, 4]})]

layer=3
E=[('abc', {}), ('bcd', {((2, 1), (2, 2)): [1, 3, 4]}), ('bce', {((2, 1), (2, 2)): [1, 4]}), ('bde', {((2, 2), (2, 2)): [1, 4]}), ('cde', {((1, 2), (1, 2)): [1, 4]})]
F=[('bcd', {((2, 1), (2, 2)): [1, 3, 4]}), ('bce', {((2, 1), (2, 2)): [1, 4]}), ('bde

['ad', 'ae', 'abc']